# BWSI UAV Course
## Coordinate Frames & Drone Dynamics

This notebook accompanies the **Controls Fundamentals** lecture (kinematics & dynamics).  
Work through the parts in order; each builds practical skills for flying‐robot control.

*Estimated effort:* 2 h in lab + 1 h write‑up

---


### Learning goals
1. Practicing Coordinate Transforms
2. Implement and chain 3‑D frame transforms with Euler angles, rotation matrices and quaternions.  
3. Convert sensor‑frame measurements into world‑frame velocity commands.  
4. Size rotor thrust for hover, climb and level‑flight cruise using the point‑mass model.  
5. Compute required pitch moment for a fast attitude step with the rigid‑body equation.


## Part 0 - Practicing Coordinate Transforms
In this Section we will be practicing how to transform vectors between the various reference frames used for navigating and controlling an autonomous quadrotor. For this work we will be relying heavily on the tf/transformations.py library.

In [1]:
!pip install transformations


In [ ]:
from __future__ import division, print_function
import numpy as np
import transformations as tft

# Variable Notation:
# v__x: vector expressed in "x" frame
# q_x_y: quaternion of "x" frame with respect to "y" frame
# p_x_y__z: position of "x" frame with respect to "y" frame expressed in "z" coordinates
# v_x_y__z: velocity of "x" frame with respect to "y" frame expressed in "z" coordinates
# R_x2y: rotation matrix that maps vector represented in frame "x" to representation in frame "y" (right-multiply column vec)
#
# Frame Subscripts:
# dc = downward-facing camera (body-fixed, non-inertial frame. Origin: downward camera focal plane. Alignment with respect to drone airframe: x-forward, y-right, z-down)
# fc = forward-facing camera (body-fixed, non-inertial frame. Origin: forward camera focal plane. Alignment with respect to drone airframe: x-right, y-down, z-forward)
# bu = body-up frame (body-fixed, non-inertial frame. Origin: drone center of mass. Alignment with respect to drone airframe: x-forward, y-left, z-up)
# bd = body-down frame (body-fixed, non-inertial frame. Origin: drone center of mass. Alignment with respect to drone airframe: x-forward, y-right, z-down)
# lenu = local East-North-Up world frame (world-fixed, inertial frame. Origin: apprx at take-off point, but not guaranteed. Alignment with respect to world: x-East, y-North, z-up)
# lned = local North-East-Down world frame (world-fixed, inertial frame. Origin: apprx at take-off point, but not guaranteed. Alignment with respect to world: x-North, y-East, z-down)
# m = marker frame (inertial or non-inertial, depending on motion of marker. Origin: center of marker. Alignment when looking at marker: x-right, y-up, z-out of plane toward you)

###Velocities of Relative Frames
Just based on your understanding of what the different reference frames represent, can you find the following velocities just by inspection? Imagine there is a ball b and a drone oriented northwest. You are given that v_b_bu_fc=[1.0, 0.0, 0.0].

1. v_b_bu_dc =
2. v_b_bu_bu =
3. v_b_bu_lned =

In [4]:
# Place your answers here in markdown or comment form

# YOUR CODE HERE
#v_b_bu_dc = [0,1,0]
#v_b_bu_bu = [0,-1,0]
#v_b_bu_lned = [-1/root2, -1/root2, 0]

###Valid Operations with Relative Frames
Which of these operations are valid and which are invalid. If valid, write the resulting variable name. If invalid, give a brief explanation why. The first is filled out for you to show answer format:



1. p_m_fc__lenu - p_bu_fc__lenu => p_m_bu__lenu
2. np.dot(R_bu2lenu, v_bu_lenu__bu) =>
3. np.dot(R_bu2lenu, v_bu_lenu__lenu) => invalid, R_bu2lenu maps a vector expressed in bu to a vector expressed in lenu, but v_bu_lenu__lenu is already expressed in lenu therefore this will generate a non-sensical answer
4. v_dc_m__dc - p_m_lenu__dc =>
5. p_bu_fc__lned - np.dot(R_m2lned, p_m_fc__m) =>







In [5]:
# Place your answers here in markdown or comment form

# YOUR CODE HERE
  # p_m_fc__lenu - p_bu_fc__lenu => p_m_bu__lenu
  # np.dot(R_bu2lenu, v_bu_lenu__bu) => v_bu_lenu_lenu
  # np.dot(R_bu2lenu, v_bu_lenu__lenu) => invalid, R_bu2lenu maps a vector expressed in bu to a vector expressed in lenu, but v_bu_lenu__lenu is already expressed in lenu therefore this will generate a non-sensical answer
  # v_dc_m__dc - p_m_lenu__dc => invalid, subtracting position from velocity
  # p_bu_fc__lned - np.dot(R_m2lned, p_m_fc__m) => p_bu_fc_lned - p_m_fc_lned => p_bu_m_lned

###Fixed/Static Relative Rotations

These rotation matrices are constant, they don't change regardless of the motion of the quadrotor. We can use this knowledge to "hard code" a set of transformations into a class we call `StaticTransforms` which can be used throughout our flight code.

In the next code block, you will need to complete some of the components and variable definitions of the `StaticTransforms` class.

In [10]:
class StaticTransforms():
    def __init__(self):
        # local ENU to local NED
        self.R_lenu2lned = np.array([[0.0, 1.0, 0.0, 0.0],
                                     [1.0, 0.0, 0.0, 0.0],
                                     [0.0, 0.0, -1.0, 0.0],
                                     [0.0, 0.0, 0.0, 1.0]])
        # YOUR CODE HERE
        # body-up to body-down (180-degree rotation around X-axis)
        self.R_bu2bd = np.array([[1.0,0.0,0.0,0.0],
                                 [0.0, -1.0, 0.0, 0.0],
                                 [0.0,0.0,-1.0,0.0],
                                 [0.0,0.0,0.0,1.0]])
        # forward camera to body-down 
        self.R_fc2bd = np.array([[ 0.0, 0.0, 1.0, 0.0],
                                 [ 0.0, 1.0, 0.0, 0.0],
                                 [-1.0, 0.0, 0.0, 0.0],
                                 [ 0.0, 0.0, 0.0, 1.0]])
        # downward camera to body-down (identity)
        self.R_dc2bd = np.eye(4)

        # Derived inverse transforms
        self.R_lned2lenu = self.R_lenu2lned.T
        self.R_bd2bu = self.R_bu2bd.T
        self.R_bd2fc = self.R_fc2bd.T
        self.R_bd2dc = self.R_dc2bd.T

        # Find concatenated rotation matrices from downward-camera to forward-camera
        self.R_dc2fc = tft.concatenate_matrices(self.R_bd2fc, self.R_dc2bd)
        self.R_fc2dc = self.R_dc2fc.T

        self.R_fc2bu = tft.concatenate_matrices(self.R_bd2bu,self.R_fc2bd)
        self.R_bu2fc = self.R_fc2bu.T

        self.R_dc2bu = tft.concatenate_matrices(self.R_bd2bu, self.R_dc2bd)
        self.R_bu2dc = self.R_dc2bu.T

    def coord_transform(self, v__fin, fin, fout):
        '''Transform vector v from fin frame to fout frame.'''
        if fin == fout:
            v4__fin = np.array(list(v__fin) + [0.0])#Adding the [0.0] is called *homogenizing* the
                                                    #vector, it allows the 3D vector to be transformed by the 4D matrix
            return v4__fin[:3]

        R_str = f'R_{fin}2{fout}'
        #check for existance of rotation matrix
        try:
            R_fin2fout = getattr(self, R_str)
        except AttributeError:
            raise AttributeError(
                f'No static transform exists from {fin} to {fout}. '
                'Are you sure these frames are not moving relative to each other?'
            )

        v4__fin = np.array(list(v__fin)+[0.0])
        v4__fout = R_fin2fout @ v4__fin
        return v4__fout[:3]

st = StaticTransforms()

In [11]:
assert np.allclose(st.coord_transform([1.0, 0.0, 0.0], 'bu', 'bu'), [1.0, 0.0, 0.0])
assert np.allclose(st.coord_transform([0.08511008, 0.38572187, 0.51372079], 'dc', 'dc'), [0.08511008, 0.38572187, 0.51372079])
assert np.allclose(st.coord_transform([0.0, 0.0, 1.0], 'fc', 'bd'), [1.0, 0.0, 0.0])
assert np.allclose(st.coord_transform([0.0, 0.0, 1.0], 'dc', 'bu'), [0.0, 0.0, -1.0])

In [15]:
# Let's assume the quadrotor has some velocity v1_bd_lned__bd which is the velocity of the quadrotor
# body-down frame with respect to the local NED world frame expressed in the body-down frame.
# Using the fixed relative rotations, calculate it's expression in the body-up, downward-camera, and forward-camera frames
v1_bd_lned__bd = [1.0, 0.0, 0.0]
# YOUR CODE HERE
transforms = StaticTransforms()
v1_bd_lned__bu = StaticTransforms.coord_transform(transforms,v1_bd_lned__bd,"bd","bu")
v1_bd_lned__dc = StaticTransforms.coord_transform(transforms,v1_bd_lned__bd,"bd","dc")
v1_bd_lned__fc = StaticTransforms.coord_transform(transforms,v1_bd_lned__bd,"bd","fc")

print(v1_bd_lned__bu)
print(v1_bd_lned__dc)
print(v1_bd_lned__fc)

# Let's assume the quadrotor has some velocity v2_bd_lned__bd which is the velocity of the quadrotor
# body-down frame with respect to the local NED world frame expressed in the body-down frame.
# Using the fixed relative rotations, calculate it's expression in the body-up, downward-camera, and forward-camera frames
v2_bd_lned__bd = [0.147, 0.798, 1.221]

v2_bd_lned__bu = StaticTransforms.coord_transform(transforms,v2_bd_lned__bd,"bd","bu")
v2_bd_lned__dc = StaticTransforms.coord_transform(transforms,v2_bd_lned__bd,"bd","dc")
v2_bd_lned__fc = StaticTransforms.coord_transform(transforms,v2_bd_lned__bd,"bd","fc")

print(v2_bd_lned__bu)
print(v2_bd_lned__dc)
print(v2_bd_lned__fc)

# Let's assume the quadrotor has some velocity v3_dc_lenu__dc which is the velocity of the quadrotor
# downward-camera frame with respect to the local ENU world frame expressed in the downward-camera frame.
# Using the static transforms, calculate it's expression in the body-down, forward-camera, and body-up frames
v3_dc_lenu__dc = [4.853, 2.979, 1.884]
# YOUR CODE HERE
v3_dc_lenu__bd = StaticTransforms.coord_transform(transforms,v3_dc_lenu__dc,"dc","bd")
v3_dc_lenu__fc = StaticTransforms.coord_transform(transforms,v3_dc_lenu__dc,"dc","fc")
v3_dc_lenu__bu = StaticTransforms.coord_transform(transforms,v3_dc_lenu__dc,"dc","bu")


print(v3_dc_lenu__bd)
print(v3_dc_lenu__fc)
print(v3_dc_lenu__bu)

# Let's assume the quadrotor has some velocity v4_fc_lenu__bd which is the velocity of the quadrotor
# forward-camera frame with respect to the local ENU world frame expressed in the body-down frame.
# Using the static transforms, calculate it's expression in the forward-camera, downward-camera and and body-up frames
v4_fc_lenu__bd = [0.0, 0.0, -1.0]
# YOUR CODE HERE

v4_fc_lenu__fc = StaticTransforms.coord_transform(transforms,v4_fc_lenu__bd,"bd","fc")
v4_fc_lenu__dc = StaticTransforms.coord_transform(transforms,v4_fc_lenu__bd,"bd","dc")
v4_fc_lenu__bu = StaticTransforms.coord_transform(transforms,v4_fc_lenu__bd,"bd","bu")

print(v4_fc_lenu__fc)
print(v4_fc_lenu__dc)
print(v4_fc_lenu__bu)


[1. 0. 0.]
[1. 0. 0.]
[0. 0. 1.]
[ 0.147 -0.798 -1.221]
[0.147 0.798 1.221]
[-1.221  0.798  0.147]
[4.853 2.979 1.884]
[-1.884  2.979  4.853]
[ 4.853 -2.979 -1.884]
[1. 0. 0.]
[ 0.  0. -1.]
[0. 0. 1.]


In [16]:
# Autograder, do not modify

assert np.allclose(v1_bd_lned__bd, [1.0, 0.0, 0.0])
assert np.allclose(v2_bd_lned__bd, [0.147, 0.798, 1.221])
assert np.allclose(v3_dc_lenu__dc, [4.853, 2.979, 1.884])
assert np.allclose(v4_fc_lenu__bd, [0.0, 0.0, -1.0])

###Dynamic Relative Rotations
In the previous section we looked at reference frames that remain fixed relative to one another (i.e. reference frames that are all attached to quadrotor or reference frames, or reference frames that are all associated with inertial local world frames). Now were going to look at reference frames that may be moving relative to one another, such as a body-fixed frame and the local world frame.

For such moving frames, we often can't create rotation matrices by inspection. Furthermore, such rotations need to be calculated automatically by the quadrotor's flight computer in real-time. This is the job of the state estimator that runs onboard the flight computer. The state estimator will output estimates of the relative rotations between local world frame and the body frame.

More specifically, the topic `mavros/local_position/pose` provides [PoseStamped](https://docs.ros.org/en/melodic/api/geometry_msgs/html/msg/PoseStamped.html) messages that contain the orientation of the body-down frame with respect to the local ENU frame in the form of a [Quaternion](https://docs.ros.org/en/melodic/api/geometry_msgs/html/msg/Quaternion.html).

Therefore, when using MAVROS, you could use an assignment such as the one below to find q_bu_lenu:

`q_bu_lenu = pose_stamped_msg.pose.orientation`

Below is a function that we can use when flying the drone to transform vectors in an arbitrary reference frame to the local ENU reference frame, assuming that we have access to the `mavros/local_position/pose` topic to tell us `q_bu_lenu` (in this case we assume they are velocity vectors)

In [17]:
def get_lenu_velocity(q_bu_lenu, v__fin, fin='bu', static_transforms=None):
        '''tranforms a vector represented in fin frame to vector in lenu frame
        Args:
        - v__fin: 3D vector represented in input frame coordinates
        - fin: string describing input coordinate frame (bd, bu, fc, dc)
        Returns:
        - v__lenu: 3D vector v represented in local ENU world frame
        '''

        # create static transforms if none given
        if static_transforms is None:
            static_transforms = StaticTransforms()

        if fin=='lenu':
            v__lenu = v__fin

        elif fin=='lned':
            v__lenu = static_transforms.coord_transform(v__fin, 'lned', 'lenu')

        else:
            # create rotation matrix from quaternion
            R_bu2lenu = tft.quaternion_matrix(q_bu_lenu)

            # represent vector v in body-down coordinates
            v__bu = static_transforms.coord_transform(v__fin, fin, 'bu')

            # calculate lenu representation of v
            v__lenu = np.dot(R_bu2lenu, list(v__bu)+[0.0])

        v__lenu = np.array(v__lenu[0:3])
        return v__lenu

###ROS tf2 Library
The problems we have attempted to address in this module (i.e. managing multiple reference frames) are by no means unique to quadrotors and we are not the first people to write functions to solve such problems. The functionality of managing different reference frames is ubiquitous throughout robotics, aerospace engineering, mechanical engineering, computer graphics, etc. and many libraries have been written for handling such functionality. When working with ROS, the most important of such libraries is the  [tf (now tf2) library](https://wiki.ros.org/tf2). While we have access to this library on the drone, we have not made use of it here because it obscures some of the underlying mathematics that we hope for you to learn and it requires additional setup steps when defining new frames that we don't intend to teach. If you are curious to know more about how ROS manages large numbers of reference frames simultaneously, we encourage you to read up more on `tf`.

NOTE: `tf` in the context of ROS should not be confused with TensorFlow which is often abbreviated as tf in code. These libraries have completely different purposes.

## Part A  Frame‑transform toolkit

In [19]:
#import numpy as np
import sympy as sp
from typing import Tuple
#import matplotlib.pyplot as plt
# from scipy.spatial.transform import Rotation as SciRot

# ---- Constants ----
GRAV = 9.81      # m s⁻²
MASS = 1.0       # kg
K_F  = 6.0e-6    # N·s²·rad⁻²

np.set_printoptions(precision=4, suppress=True)


In [20]:
def euler_to_rot(roll: float, pitch: float, yaw: float, order: str = "xyz") -> np.ndarray:
    """Return a 3×3 rotation matrix given Euler angles (rad)."""
    c, s = np.cos, np.sin
    Rx = np.array([[1,0,0],[0,c(roll),-s(roll)],[0,s(roll),c(roll)]])
    Ry = np.array([[c(pitch),0,s(pitch)],[0,1,0],[-s(pitch),0,c(pitch)]])
    Rz = np.array([[c(yaw),-s(yaw),0],[s(yaw),c(yaw),0],[0,0,1]])
    mats = {"x":Rx, "y":Ry, "z":Rz}
    R = np.eye(3)
    for ax in order:
        R = R @ mats[ax]
    return R


In [21]:
def rot_to_quat(R: np.ndarray) -> np.ndarray:
    """Rotation matrix -> quaternion (scalar‑last)."""
    q = np.empty(4)
    t = np.trace(R)
    if t > 0:
        q[3] = np.sqrt(1 + t) / 2
        q[0] = (R[2,1] - R[1,2]) / (4*q[3])
        q[1] = (R[0,2] - R[2,0]) / (4*q[3])
        q[2] = (R[1,0] - R[0,1]) / (4*q[3])
    else:
        i = np.argmax(np.diag(R))
        j, k = (i+1)%3, (i+2)%3
        q[i] = np.sqrt(1 + R[i,i] - R[j,j] - R[k,k]) / 2
        q[3] = (R[k,j] - R[j,k]) / (4*q[i])
        q[j] = (R[j,i] + R[i,j]) / (4*q[i])
        q[k] = (R[k,i] + R[i,k]) / (4*q[i])
    return q / np.linalg.norm(q)

def quat_to_rot(q: np.ndarray) -> np.ndarray:
    q = q / np.linalg.norm(q)
    x, y, z, w = q
    return np.array([
        [1-2*(y**2+z**2), 2*(x*y - z*w),   2*(x*z + y*w)],
        [2*(x*y + z*w),   1-2*(x**2+z**2), 2*(y*z - x*w)],
        [2*(x*z - y*w),   2*(y*z + x*w),   1-2*(x**2+y**2)]
    ])


In [22]:
# Example transform: camera frame -> world------YOUR CODE HERE
v_dc = np.array([0.1, 0.0, 1.0])
R_dc2bu = euler_to_rot(np.deg2rad(0), np.deg2rad(-90), 0)
R_bu2enu = euler_to_rot(0, 0, np.deg2rad(45))
v_enu = R_bu2enu @ R_dc2bu @ v_dc
v_enu /= np.linalg.norm(v_enu)
print("World‑frame velocity:", v_enu)


World‑frame velocity: [-0.7036 -0.7036  0.0995]


In [23]:
R = euler_to_rot(0.3, -0.2, 1.0)
assert np.allclose(R.T @ R, np.eye(3))
q = rot_to_quat(R)
assert np.allclose(quat_to_rot(q), R)
print("Frame‑transform sanity checks passed ✔️")


Frame‑transform sanity checks passed ✔️


## Part B  Point‑mass thrust sizing

In [27]:
def thrust_allocation(m: float, k_f: float, T_total: float) -> Tuple[float, float]:
    per = T_total/4
    return np.sqrt(per/k_f), per

T_hover = MASS*GRAV
ω_hov, T_per = thrust_allocation(MASS, K_F, T_hover)
print(f"Hover ω {ω_hov:.0f} rad/s")

#TODO: calculate w to climb at 1 m/s^2
T_climb = MASS*(GRAV+1)
ω_climb, T_per = thrust_allocation(MASS, K_F, T_climb)
print(f"Climb ω {ω_climb:.0f} rad/s")

#TODO: calculate w when pitched at 30 degrees
T_pitch = MASS*(GRAV)/np.cos(np.radians(30))
ω_pitch, T_per = thrust_allocation(MASS, K_F, T_pitch)
print(f"Pitch ω {ω_pitch:.0f} rad/s")

Hover ω 639 rad/s
Climb ω 671 rad/s
Pitch ω 687 rad/s


## Part C  Rigid‑body attitude step

In [34]:
l,d,h = 0.3,0.2,0.1
I_y = 1/12*MASS*np.array([[h**2+d**2,0.0,0.0],
                     [0.0,h**2+l**2,0.0],
                     [0.0,0.0,l**2+d**2]])
theta = np.deg2rad(11.2); t=0.1
alpha = 2* theta / t**2
ω_max = alpha*t
M_y = I_y[1][1]*alpha
print(f"I_y {I_y[1][1]:.4f} kg·m², α {alpha:.1f} rad/s², moment {M_y:.3f} N·m")


I_y 0.0083 kg·m², α 39.1 rad/s², moment 0.326 N·m


---

*Happy flying!* ✈️
